<b><font size="5">Import relevant libraries and dataframes</font><a class="anchor"><a id='toc'></a></b>

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import warnings
warnings.simplefilter("ignore")
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import Birch
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score, calinski_harabasz_score

In [2]:
wm_cleaned = pd.read_csv(r'WineMatters_cleaned.csv')
wm_cleaned.set_index('WineID', inplace = True)
wm_cleaned.head()

,Aging_Time,Litters_Barrel,Magnesium,Residual_Sugar,Acidity,Floral,Wood,Sweetness,Red_Fruit,Citric,...,Alcohol,Astringency,Satisfaction_France,Satisfaction_Spain,Satisfaction_Portugal,Sulphites_Presence,Type_White,Barrel_Wooden,Grapes_Single,Satisfaction_General
WineID,,,,,,,,,,,,,,,,,,,,,
1,65,265,0,0.0,0,5,0,2,2,4,...,0,0,3,3,2,0,1,0,1,1
2,47,2464,310,305.0,4,2,0,3,0,2,...,0,0,4,3,2,1,0,0,1,1
3,15,2138,0,0.0,3,4,0,3,2,0,...,0,0,4,4,2,0,1,0,1,1
4,60,623,0,0.0,0,4,0,3,3,4,...,0,0,1,1,3,0,1,0,1,0
5,70,354,0,0.0,0,4,0,3,4,3,...,0,0,2,2,5,0,1,0,1,1


In [3]:
wm_scaled = pd.read_csv(r'WineMatters_scaled.csv')
wm_scaled.set_index('Unnamed: 0', inplace = True)
wm_scaled.head()

,Aging_Time,Litters_Barrel,Magnesium,Residual_Sugar,Acidity,Floral,Wood,Sweetness,Red_Fruit,Citric,...,Cloudiness,Alcohol,Astringency,Satisfaction_France,Satisfaction_Spain,Satisfaction_Portugal,Sulphites_Presence,Type_White,Barrel_Wooden,Grapes_Single
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
1,0.743590,0.034003,0.000000,0.000000,0.0,5.0,0.0,2.0,2.0,4.0,...,3.0,0.0,0.0,3.0,3.0,2.0,0.0,1.0,0.0,1.0
2,0.512821,0.381781,0.707763,0.693182,4.0,2.0,0.0,3.0,0.0,2.0,...,4.0,0.0,0.0,4.0,3.0,2.0,1.0,0.0,0.0,1.0
3,0.102564,0.330223,0.000000,0.000000,3.0,4.0,0.0,3.0,2.0,0.0,...,3.0,0.0,0.0,4.0,4.0,2.0,0.0,1.0,0.0,1.0
4,0.679487,0.090622,0.000000,0.000000,0.0,4.0,0.0,3.0,3.0,4.0,...,1.0,0.0,0.0,1.0,1.0,3.0,0.0,1.0,0.0,1.0
5,0.807692,0.048078,0.000000,0.000000,0.0,4.0,0.0,3.0,4.0,3.0,...,2.0,0.0,0.0,2.0,2.0,5.0,0.0,1.0,0.0,1.0


In [4]:
wm_flavfeel = pd.read_csv(r'WineMatters_flavfeel.csv')
wm_flavfeel.set_index('WineID', inplace = True)
wm_flavfeel.head()

,Residual_Sugar,Acidity,Floral,Wood,Sweetness,Red_Fruit,Citric,Density,Cloudiness,Alcohol,Astringency
WineID,,,,,,,,,,,
1,0.0,0,5,0,2,2,4,2,3,0,0
2,305.0,4,2,0,3,0,2,2,4,0,0
3,0.0,3,4,0,3,2,0,2,3,0,0
4,0.0,0,4,0,3,3,4,3,1,0,0
5,0.0,0,4,0,3,4,3,4,2,0,0


In [5]:
wm_prodchar = pd.read_csv(r'WineMatters_prodchar.csv')
wm_prodchar.set_index('WineID', inplace = True)
wm_prodchar.head()

,Aging_Time,Litters_Barrel,Type_White,Magnesium,Sulphites_Presence,Barrel_Wooden,Grapes_Single,Color_Intensity
WineID,,,,,,,,
1,65,265,1,0,0,0,1,3
2,47,2464,0,310,1,0,1,3
3,15,2138,1,0,0,0,1,2
4,60,623,1,0,0,0,1,1
5,70,354,1,0,0,0,1,2


<b><font size="5">BIRCH - wm_flavfeel</font><a class="anchor"><a id='toc'></a></b>

<b><font size="5">###Discover If we should use scale ornot scaled####</font><a class="anchor"><a id='toc'></a></b>

In [6]:
#We make a sample of the data set because it is to big to apply the algorithm 
wm_flavfeel_sample=wm_flavfeel.sample(frac=0.4, replace=True, random_state=1)
wm_flavfeel_sample

,Residual_Sugar,Acidity,Floral,Wood,Sweetness,Red_Fruit,Citric,Density,Cloudiness,Alcohol,Astringency
WineID,,,,,,,,,,,
128605,0.0,2,3,2,2,5,5,5,5,2,2
98991,11.0,2,5,2,2,4,4,4,5,2,2
78069,70.0,2,3,1,2,4,2,3,3,3,2
5244,14.0,2,3,1,1,3,2,3,2,2,1
98499,20.0,1,3,1,1,4,4,4,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...
71567,2.0,2,3,3,3,4,1,1,2,2,3
93657,0.0,4,5,4,4,5,4,5,4,4,4
107391,48.0,4,4,2,1,3,5,4,4,1,1


In [7]:
wm_flavfeel_sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51724 entries, 128605 to 117475
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Residual_Sugar  51724 non-null  float64
 1   Acidity         51724 non-null  int64  
 2   Floral          51724 non-null  int64  
 3   Wood            51724 non-null  int64  
 4   Sweetness       51724 non-null  int64  
 5   Red_Fruit       51724 non-null  int64  
 6   Citric          51724 non-null  int64  
 7   Density         51724 non-null  int64  
 8   Cloudiness      51724 non-null  int64  
 9   Alcohol         51724 non-null  int64  
 10  Astringency     51724 non-null  int64  
dtypes: float64(1), int64(10)
memory usage: 4.7 MB


In [8]:
bclust = Birch(branching_factor=50, n_clusters=3, threshold=.5).fit(wm_flavfeel_sample)

MemoryError: Unable to allocate 4.80 GiB for an array with shape (644135778,) and data type float64

In [ ]:
bclust.subcluster_centers_

In [ ]:
bclust.subcluster_labels_

In [ ]:
labels = bclust.predict(wm_flavfeel_sample)
labels

In [ ]:
wm_flavfeel_BIRCH=wm_flavfeel_sample

In [ ]:
wm_flavfeel_res = wm_flavfeel_BIRCH.copy()

In [ ]:
wm_flavfeel_res['label_b'] = labels

In [ ]:
descr = wm_flavfeel_res.reset_index(drop=True)
descr = descr.groupby(['label_b']).describe().transpose()
descr

In [ ]:
kclust = KMeans(n_clusters=3)

In [ ]:
bclust=Birch(branching_factor=50, n_clusters=kclust, threshold=.5).fit(wm_flavfeel_sample_scalled)
labels = bclust.predict(wm_flavfeel_sample_scalled)
wm_flavfeel_res['label_bk'] = labels

In [ ]:
descr_bk = wm_flavfeel_res.reset_index(drop=True).drop(columns=['label_b'])
descr_bk = descr_bk.groupby(['label_bk']).describe().transpose()
descr_bk

<b><font size="5">BIRCH - wm_prodchar</font><a class="anchor"><a id='toc'></a></b>

In [ ]:
#We make a sample of the data set because it is to big to apply the algorithm 
wm_prodchar_sample=wm_prodchar.sample(frac=0.5, replace=True, random_state=1)
wm_prodchar_sample

In [ ]:
minmax_scaler = MinMaxScaler()

In [ ]:
scale = minmax_scaler.fit(wm_prodchar_sample)

In [ ]:
minmax_data = scale.transform(wm_prodchar_sample)

In [ ]:
wm_prodchar_sample_scalled = pd.DataFrame(minmax_data, columns = wm_prodchar_sample.columns)
wm_prodchar_sample_scalled.head()

In [ ]:
wm_prodchar_sample_scalled.info()

In [ ]:
bclust_prodchar = Birch(branching_factor=50, n_clusters=3, threshold=.5).fit(wm_prodchar_sample_scalled)

In [ ]:
bclust_prodchar.subcluster_centers_

In [ ]:
bclust_prodchar.subcluster_labels_

In [ ]:
labels = bclust_prodchar.predict(wm_prodchar_sample_scalled)
labels

In [ ]:
wm_prodchar_BIRCH=wm_prodchar_sample_scalled

In [ ]:
wm_prodchar_res = wm_prodchar_BIRCH.copy()

In [ ]:
wm_prodchar_res['label_b'] = labels

In [ ]:
descr = wm_prodchar_res.reset_index(drop=True)
descr = descr.groupby(['label_b']).describe().transpose()
descr

In [ ]:
kclust = KMeans(n_clusters=3)

In [ ]:
bclust_prodchar=Birch(branching_factor=50, n_clusters=kclust, threshold=.5).fit(wm_prodchar_sample_scalled)
labels = bclust_prodchar.predict(wm_prodchar_sample_scalled)
wm_prodchar_res['label_bk'] = labels

In [ ]:
descr_bk_prodchar = wm_prodchar_res.reset_index(drop=True).drop(columns=['label_b'])
descr_bk_prodchar = descr_bk_prodchar.groupby(['label_bk']).describe().transpose()
descr_bk_prodchar

<b><font size="5">DBSCAN - wm_prodchar</font><a class="anchor"><a id='toc'></a></b>

In [ ]:
wm_flavfeel_sample_scalled

In [ ]:
dbclust_flavfeel = DBSCAN(eps = 0.1, min_samples = 6).fit(wm_flavfeel_sample_scalled)

In [ ]:
dbclust_flavfeel.labels_

In [ ]:
len(set(dbclust_flavfeel.labels_))

In [ ]:
wm_flavfeel_res['label_db'] = dbclust_flavfeel.labels_

In [ ]:
descr_bk = wm_flavfeel_res.reset_index(drop=True).drop(columns=['label_b', 'label_bk'])
descr_bk = descr_bk.groupby(['label_db']).describe().transpose()
descr_bk

<b><font size="5">DBSCAN - wm_prodchar</font><a class="anchor"><a id='toc'></a></b>

In [ ]:
wm_prodchar_sample_scalled

In [ ]:
dbclust_prodchar = DBSCAN(eps = 0.1, min_samples = 6).fit(wm_prodchar_sample_scalled)

In [ ]:
dbclust_prodchar.labels_

In [ ]:
len(set(dbclust_prodchar.labels_))

In [ ]:
wm_prodchar_res['label_db'] = dbclust_flavfeel.labels_

In [ ]:
descr_bk = wm_prodchar_res.reset_index(drop=True).drop(columns=['label_b', 'label_bk'])
descr_bk = descr_bk.groupby(['label_db']).describe().transpose()
descr_bk

<b><font size="5">Cluster Visualization- Flavfeel</font><a class="anchor"><a id='toc'></a></b>

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4), squeeze=False)
axs[0,0].scatter(wm_flavfeel_res.Residual_Sugar, wm_flavfeel_res.Acidity, c = wm_flavfeel_res.label_b)
axs[0,0].set_title('Clustering Using BIRCH + Agglomerative')
axs[0,0].set_xlabel('Residual_Sugar')
axs[0,0].set_ylabel('Acidity')
axs[0,1].scatter(wm_flavfeel_res.Residual_Sugar, wm_flavfeel_res.Acidity, c = wm_flavfeel_res.label_bk)
axs[0,1].set_title('Clustering Using BIRCH + K-Means')
axs[0,1].set_xlabel('Residual_Sugar')
axs[0,1].set_ylabel('Acidity')
axs[0,2].scatter(wm_flavfeel_res.Residual_Sugar, wm_flavfeel_res.Acidity, c = wm_flavfeel_res.label_db)
axs[0,2].set_title('Clustering Using DBSCAN')
axs[0,2].set_xlabel('Residual_Sugar')
axs[0,2].set_ylabel('Acidity')

Although all of this 3 cluster algorithms are not fitting well in our dataset the second one (Clustering Using BIRCH + Agglomerative) it seems to be the best once that the clusters are better devided.

In [ ]:
#sns.pairplot(wm_flavfeel_res.drop(columns=['label_bk', 'label_b']), hue='label_db')

<b><font size="5">Cluster Visualization- Prodchar</font><a class="anchor"><a id='toc'></a></b>

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4), squeeze=False
 )
axs[0,0].scatter(wm_prodchar_res.Aging_Time, wm_prodchar_res.Litters_Barrel, c = wm_prodchar_res.label_b)
axs[0,0].set_title('Clustering Using BIRCH + Agglomerative')
axs[0,0].set_xlabel('Aging_Time')
axs[0,0].set_ylabel('Litters_Barrel')
axs[0,1].scatter(wm_prodchar_res.Aging_Time, wm_prodchar_res.Litters_Barrel, c = wm_prodchar_res.label_bk)
axs[0,1].set_title('Clustering Using BIRCH + K-Means')
axs[0,1].set_xlabel('Aging_Time')
axs[0,1].set_ylabel('Litters_Barrel')
axs[0,2].scatter(wm_prodchar_res.Aging_Time, wm_prodchar_res.Litters_Barrel, c = wm_prodchar_res.label_db)
axs[0,2].set_title('Clustering Using DBSCAN')
axs[0,2].set_xlabel('Aging_Time')
axs[0,2].set_ylabel('Litters_Barrel')

In [ ]:
#sns.pairplot(wm_prodchar_res.drop(columns=['label_bk', 'label_b']), hue='label_db')

The DBSCAN is not he best option for our dataset once that we cannot see clearly the division of the 3 clusters.

<b><font size="5">Cluster Evaluation- Flavfeel</font><a class="anchor"><a id='toc'></a></b>

In [ ]:
def clusters_scores(X, cluster_algorithm, score_function):
    range_n_clusters = [2, 3, 4, 5, 6]
    scores =[]
    
    for n_clusters in range_n_clusters:
        # Initialize the clusterer with n_clusters value
        clusterer = cluster_algorithm(n_clusters=n_clusters)
        cluster_labels = clusterer.fit_predict(X)

        score_avg = score_function(X, cluster_labels)
        scores.append(score_avg)

        print("For n_clusters =", n_clusters, "The average", score_function.__name__, "is :", score_avg)
    
    return scores

<b><font size="3">Silhouette Score</font><a class="anchor"><a id='toc'></a></b>

In [ ]:
k_means_silhouette = clusters_scores(wm_flavfeel_sample_scalled, KMeans, silhouette_score)

In [ ]:
birch_silhouette = clusters_scores(wm_flavfeel_sample_scalled, Birch, silhouette_score)

In [ ]:
def draw_scores(scores):
    plt.figure(figsize=(5, 5))
    plt.plot(range(2, 7), scores,c="#008fd5",label="Silhouette Coefficient")

In [ ]:
draw_scores(k_means_silhouette)

In [ ]:
draw_scores(birch_silhouette)

<b><font size="3">Calinski Harabasz Score</font><a class="anchor"><a id='toc'></a></b>

In [ ]:
k_means_calinski = clusters_scores(wm_flavfeel_sample_scalled, KMeans, calinski_harabasz_score)

In [ ]:
birch_calinski = clusters_scores(wm_flavfeel_sample_scalled, Birch, calinski_harabasz_score)

In [ ]:
draw_scores(k_means_calinski)

In [ ]:
draw_scores(birch_calinski)

<b><font size="5">Cluster Evaluation- Prodchar</font><a class="anchor"><a id='toc'></a></b>

In [ ]:
def clusters_scores(X, cluster_algorithm, score_function):
    range_n_clusters = [2, 3, 4, 5, 6]
    scores =[]
    
    for n_clusters in range_n_clusters:
        # Initialize the clusterer with n_clusters value
        clusterer = cluster_algorithm(n_clusters=n_clusters)
        cluster_labels = clusterer.fit_predict(X)

        score_avg = score_function(X, cluster_labels)
        scores.append(score_avg)

        print("For n_clusters =", n_clusters, "The average", score_function.__name__, "is :", score_avg)
    
    return scores

<b><font size="3">Silhouette Score</font><a class="anchor"><a id='toc'></a></b>

In [ ]:
k_means_silhouette = clusters_scores(wm_prodchar_sample_scalled, KMeans, silhouette_score)

In [ ]:
birch_silhouette = clusters_scores(wm_prodchar_sample_scalled, Birch, silhouette_score)

In [ ]:
def draw_scores(scores):
    plt.figure(figsize=(5, 5))
    plt.plot(range(2, 7), scores,c="#008fd5",label="Silhouette Coefficient")

In [ ]:
draw_scores(k_means_silhouette)

In [ ]:
draw_scores(birch_silhouette)

<b><font size="3">Calinski Harabasz Score</font><a class="anchor"><a id='toc'></a></b>

In [ ]:
k_means_calinski = clusters_scores(wm_prodchar_sample_scalled, KMeans, calinski_harabasz_score)

In [ ]:
birch_calinski = clusters_scores(wm_prodchar_sample_scalled, Birch, calinski_harabasz_score)

In [ ]:
draw_scores(k_means_calinski)

In [ ]:
draw_scores(birch_calinski)